In [3]:
import transformers
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [4]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [5]:
df_fake.shape

(23481, 4)

In [6]:
df_true.shape

(21417, 4)

In [7]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
df_fake['fake'] = 1

In [9]:
df_true['fake'] = 0

In [10]:
df_true.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [11]:
df_final = pd.concat([df_true, df_fake]).reset_index(drop = True)

In [12]:
df_final.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [13]:
df_final.shape

(44898, 5)

In [14]:
df_final.drop(['date'], axis = 1, inplace = True)

In [15]:
df_final.head()

,title,text,subject,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,0


In [24]:
import string
from nltk.stem import WordNetLemmatizer

In [26]:
from nltk.corpus import stopwords

In [30]:
english_stopwords = stopwords.words('english')

In [31]:
def text_preprocessor(text):
    
    text_preprocessed = []
    text = text.translate(str.maketrans('', '', string.punctuation)) # Removes punctuations
    for word in text.split(' '): # Splits the text into words
        # print(word)
        word = word.lower()
        lemmatizer = WordNetLemmatizer()
        word = lemmatizer.lemmatize(word)
        if word not in english_stopwords:
            text_preprocessed.append(word)
    text_preprocessed = ' '.join(text_preprocessed)
    return text_preprocessed

In [32]:
df_final['title preprocessed'] = df_final['title'].apply(text_preprocessor)

In [34]:
df_final.drop(['title'], axis = 1, inplace = True)
df_final.head()

,text,subject,fake,title preprocessed
0,WASHINGTON (Reuters) - The head of a conservat...,politicsNews,0,u budget fight loom republican flip fiscal script
1,WASHINGTON (Reuters) - Transgender people will...,politicsNews,0,u military accept transgender recruit monday p...
2,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,0,senior u republican senator let mr mueller job
3,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,0,fbi russia probe helped australian diplomat ti...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,0,trump want postal service charge much amazon s...


In [35]:
# This can take about 5 minutes to run as we are preprocessing the entire text
df_final['text preprocessed'] = df_final['text'].apply(text_preprocessor)

In [36]:
df_final.drop(['text'], axis = 1, inplace = True)
df_final.head()

,subject,fake,title preprocessed,text preprocessed
0,politicsNews,0,u budget fight loom republican flip fiscal script,washington reuters head conservative republic...
1,politicsNews,0,u military accept transgender recruit monday p...,washington reuters transgender people allowed...
2,politicsNews,0,senior u republican senator let mr mueller job,washington reuters special counsel investigat...
3,politicsNews,0,fbi russia probe helped australian diplomat ti...,washington reuters trump campaign adviser geo...
4,politicsNews,0,trump want postal service charge much amazon s...,seattlewashington reuters president donald tr...


In [38]:
title_list = df_final['title preprocessed'].tolist()
text_list = df_final['text preprocessed'].tolist()

In [39]:
title_list[0: 5]

['u budget fight loom republican flip fiscal script',
 'u military accept transgender recruit monday pentagon',
 'senior u republican senator let mr mueller job',
 'fbi russia probe helped australian diplomat tipoff nyt',
 'trump want postal service charge much amazon shipment']

In [42]:
from transformers import pipeline

In [44]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceCla

In [45]:
classifier('This is a bad example of using transformers')

[{'label': 'NEGATIVE', 'score': 0.9997902512550354}]

In [46]:
classifier('This is a good thing to consider. However, there are other things that we have neglected')

[{'label': 'POSITIVE', 'score': 0.9914538860321045}]

In [47]:
title_list[0]

'u budget fight loom republican flip fiscal script'

In [58]:
classifier(title_list[0])

[{'label': 'NEGATIVE', 'score': 0.9936803579330444}]

In [59]:
text_list[0][0: 2000]

'washington reuters  head conservative republican faction u congress voted month huge expansion national debt pay tax cut called “fiscal conservative” sunday urged budget restraint 2018 keeping sharp pivot way among republican u representative mark meadow speaking cbs’ “face nation” drew hard line federal spending lawmaker bracing battle january return holiday wednesday lawmaker begin trying pas federal budget fight likely linked issue immigration policy even november congressional election campaign approach republican seek keep control congress president donald trump republican want big budget increase military spending democrat also want proportional increase nondefense “discretionary” spending program support education scientific research infrastructure public health environmental protection “the trump administration ha already willing say ‘we’re going increase nondefense discretionary spending  7 percent’” meadow chairman small influential house freedom caucus said program “now dem

In [57]:
classifier(text_list[0][0: 2000])

[{'label': 'NEGATIVE', 'score': 0.9760938286781311}]

In [60]:
classifier = pipeline('sentiment-analysis', model = 'bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [61]:
from transformers import BertTokenizer

In [62]:
classifier('This is not a good sentence')

[{'label': 'LABEL_1', 'score': 0.5203396081924438}]

In [63]:
from transformers import AutoTokenizer, AutoModelForMaskedLM